# Modelling

In [47]:
# match data with aggregated individual data
import pandas as pd
match_path = '/Users/t_raver9/Desktop/projects/aflengine/analysis/machine_learning/src/player_data/data/matches_with_player_agg.csv'
players_path = '/Users/t_raver9/Desktop/projects/aflengine/analysis/machine_learning/src/player_data/data/players_with_player_stat_totals.csv'
matches = pd.read_csv(match_path)
players = pd.read_csv(players_path)

/Applications/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (16,21,28) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## Data Preparation

For the first iteration, we'll only use data after 2009. This is when most modern statistics began to be kept (though not all of them did).

In [48]:
model_data = matches[matches['season'] >= 2010]

To keep model simple, exclude draws. Mark them as victories for the away team instead.

In [237]:
for idx, row in model_data.iterrows():
    if row['winner'] == 'draw':
        model_data.at[idx,'winner'] = 'away'

We want to split the data into test and train in a stratified manner, i.e. we don't want to favour a certain season, or a part of the season. So we'll take a portion (25%) of games from each round.

In [49]:
# How many games do we get per round?
round_counts = {}
curr_round = 1
matches_in_round = 0
for idx,row in model_data.iterrows():
    
    if curr_round != row['round']:
        
        if matches_in_round not in round_counts:
            round_counts[matches_in_round] = 1
        else:
            round_counts[matches_in_round] += 1
            
        curr_round = row['round']
        matches_in_round = 1
        continue
        
    else:
        matches_in_round += 1
        
round_counts

{8: 44, 7: 7, 9: 157, 6: 22}

In [50]:
# Taking a minimum 25% of each round
from math import ceil
test_sample_size = {}
for num_games in round_counts:
    test_sample_size[num_games] = ceil(num_games/4)

In [51]:
rounds_in_season = get_season_rounds(model_data)
teams_in_season = get_season_teams(model_data)

Create test and training data

In [238]:
# test set
from copy import deepcopy

test_data = pd.DataFrame()
for season, max_round in rounds_in_season.items():
    for rnd in range(1, max_round):
        round_matches = model_data[(model_data['season']==season) & (model_data['round']==rnd)]
        num_test = test_sample_size[len(round_matches)]
        round_test_set = round_matches.sample(num_test)
        test_data = test_data.append(round_test_set)
        
# training set
training_data = model_data.drop(test_data.index)

Define features

In [239]:
features = [col 
            for col 
            in ['h_career_' + col for col in player_cols_to_agg] + \
            ['h_season_' + col for col in player_cols_to_agg] + \
            ['a_career_' + col for col in player_cols_to_agg] + \
            ['a_season_' + col for col in player_cols_to_agg] + \
            ['h_' + col for col in ladder_cols] + \
            ['h_' + col + '_form' for col in ladder_cols] + \
            ['a_' + col for col in ladder_cols] + \
            ['a_' + col + '_form' for col in ladder_cols] + \
            ['h_career_' + col for col in misc_columns] + \
            ['h_season_' + col for col in misc_columns] + \
            ['a_career_' + col for col in misc_columns] + \
            ['a_season_' + col for col in misc_columns]
            ]

target = 'winner'

Set up test and train datasets

In [240]:
X_train = training_data[features]
y_train = training_data[target]
X_test = test_data[features]
y_test = test_data[target]

Fill the NaN values

In [241]:
X_train.fillna(0,inplace=True)
y_train.fillna(0,inplace=True)
X_test.fillna(0,inplace=True)
y_test.fillna(0,inplace=True)

/Applications/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4244: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  **kwargs


## Modelling

Model 1: Logistic regression

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
import numpy as np

log_reg = LogisticRegression()

param_grid = {
                 'tol': [.0001, .001, .01],
                 'C': [.1, 1, 10],
                 'max_iter': [50,100,200]
             }

grid_log_reg = GridSearchCV(log_reg, param_grid, cv=5)
grid_log_reg.fit(X_train, y_train)

/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to '

/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to '

/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to '

/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lb

In [58]:
grid_log_reg.score(X_train,y_train)

0.7415812591508053

In [59]:
grid_log_reg.score(X_test,y_test)

0.6960950764006791

In [218]:
# Confirm that it's not just picking the home team
print(sum(grid_log_reg.predict(X_test)=='away'))
print(sum(grid_log_reg.predict(X_test)=='home'))

241
348


## Generating predictions

Now that we have a model, we need to ingest data for that model to make a prediction on.

Start by reading in the fixture.

In [60]:
fixture_path = '/Users/t_raver9/Desktop/projects/aflengine/tipengine/fixture2020.csv'
fixture = pd.read_csv(fixture_path)

In [61]:
fixture[fixture['round']=='2']

,Unnamed: 0,round,date,venue,hometeam,awayteam,homescore,awayscore,winner
9,9,2,26/03/2020 19:25,MCG,Collingwood,Richmond,NaN,NaN,NaN
10,10,2,27/03/2020 19:50,SCG,Sydney,Essendon,NaN,NaN,NaN
11,11,2,28/03/2020 13:45,MCG,Melbourne,Greater Western Sydney,NaN,NaN,NaN
12,12,2,28/03/2020 16:35,Marvel Stadium,Carlton,Footscray,NaN,NaN,NaN
13,13,2,28/03/2020 19:25,Gabba,Brisbane Lions,North Melbourne,NaN,NaN,NaN
14,14,2,28/03/2020 19:40,Adelaide Oval,Port Adelaide,Adelaide,NaN,NaN,NaN
15,15,2,29/03/2020 13:10,GMHBA Stadium,Geelong,Gold Coast,NaN,NaN,NaN
16,16,2,29/03/2020 15:20,Marvel Stadium,St Kilda,West Coast,NaN,NaN,NaN
17,17,2,29/03/2020 18:20,Optus Stadium,Fremantle,Hawthorn,NaN,NaN,NaN


We'll then prepare the data for the round we're interested in. We'll do this by:
- getting the team-level data, such as ladder position and form
- getting the player-level data and aggregating it up to the team level

To get the player-level data, we also need to choose who is playing for each team.

In [62]:
next_round_matches = get_upcoming_matches(matches,fixture,round_num=2)

/Applications/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:7123: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


In [63]:
next_round_matches

,a_draws,a_draws_form,a_losses,a_losses_form,a_percentage,a_percentage_form,a_played,a_played_form,a_points_against,a_points_against_form,...,h_points_against_form,h_points_for,h_points_for_form,h_position,h_position_form,h_prem_points,h_prem_points_form,h_wins,h_wins_form,hteam
0,0,0,0,-6,129.63,103.3,0,-16,81,-1212,...,-1130,86,-1278,1,-1,4,-40,1,-10,Collingwood
0,0,0,0,-7,110.526,101.979,0,-16,57,-1162,...,-1224,74,-1160,1,-13,4,-20,1,-5,Sydney
0,0,0,0,-7,143.836,119.66,0,-16,73,-1119,...,-1376,51,-1117,1,-15,0,-20,0,-5,Melbourne
0,0,0,1,-7,39.5349,98.5703,0,-16,86,-1259,...,-1296,81,-1083,1,-16,0,-16,0,-4,Carlton
0,0,0,0,-9,103.704,99.3066,0,-16,54,-1298,...,-1180,62,-1396,1,-2,0,-44,0,-11,Brisbane Lions
0,0,0,1,-6,95.9459,113.886,0,-16,74,-1109,...,-1194,76,-1215,1,-8,4,-28,1,-7,Port Adelaide
0,0,0,1,-12,38.1579,65.4646,0,-16,76,-1442,...,-947,73,-1410,1,0,0,-52,0,-13,Geelong
0,0,0,0,-5,152.941,111.559,0,-16,51,-1142,...,-1383,54,-1097,1,-14,0,-24,0,-6,St Kilda
0,0,0,0,-9,145.161,97.6764,0,-16,62,-1162,...,-1161,57,-1118,1,-12,0,-28,0,-7,Fremantle


Get the IDs for the players we'll be using

In [70]:
import cv2 
import pytesseract
custom_config = r'--oem 3 --psm 6'

In [71]:
import pathlib
names_dir = '/Users/t_raver9/Desktop/projects/aflengine/analysis/machine_learning/src/OCR/images'

# Initialise the dictionary
player_names_dict = {}
for team in matches['hteam'].unique():
    player_names_dict[team] = []
    
# Fill out the dictionary
for path in pathlib.Path(names_dir).iterdir():
    print(path)
    if path.name.split('.')[0] in player_names_dict:
        path_str = str(path)
        image_obj = cv2.imread(path_str)
        image_string = pytesseract.image_to_string(image_obj, config=custom_config)
        names = get_player_names(image_string)
        player_names_dict[path.name.split('.')[0]].extend(names)

/Users/t_raver9/Desktop/projects/aflengine/analysis/machine_learning/src/OCR/images/St Kilda.png
/Users/t_raver9/Desktop/projects/aflengine/analysis/machine_learning/src/OCR/images/Hawthorn.png
/Users/t_raver9/Desktop/projects/aflengine/analysis/machine_learning/src/OCR/images/Adelaide.png
/Users/t_raver9/Desktop/projects/aflengine/analysis/machine_learning/src/OCR/images/.DS_Store
/Users/t_raver9/Desktop/projects/aflengine/analysis/machine_learning/src/OCR/images/Gold Coast.png
/Users/t_raver9/Desktop/projects/aflengine/analysis/machine_learning/src/OCR/images/Essendon.png
/Users/t_raver9/Desktop/projects/aflengine/analysis/machine_learning/src/OCR/images/Collingwood.png
/Users/t_raver9/Desktop/projects/aflengine/analysis/machine_learning/src/OCR/images/North Melbourne.png
/Users/t_raver9/Desktop/projects/aflengine/analysis/machine_learning/src/OCR/images/Geelong.png
/Users/t_raver9/Desktop/projects/aflengine/analysis/machine_learning/src/OCR/images/Melbourne.png
/Users/t_raver9/Deskt

Now we can collect the data for each player and aggregate it to the team level, as we would with the training data

In [92]:
from copy import deepcopy

players_in_rnd = []
for _, v in player_names_dict.items():
    players_in_rnd.extend(v)

player_data = get_player_data(players_in_rnd)

In [196]:
aggregate = player_data[player_cols].groupby('team').apply(lambda x: x.mean(skipna=False))
aggs_h = deepcopy(aggregate)
aggs_a = deepcopy(aggregate)
aggs_h.columns = aggregate.columns.map(lambda x: 'h_' + str(x))
aggs_a.columns = aggregate.columns.map(lambda x: 'a_' + str(x))
combined = next_round_matches.merge(aggs_h, left_on='hteam', right_on='team')
combined = combined.merge(aggs_a, left_on='hteam', right_on='team')

In [197]:
combined

,a_draws,a_draws_form,a_losses,a_losses_form,a_percentage,a_percentage_form,a_played,a_played_form,a_points_against,a_points_against_form,...,a_season_disposal_efficiency,a_season_effective_disposals,a_season_intercepts,a_season_metres_gained,a_season_stoppage_clearances,a_season_score_involvements,a_season_tackles_in_50,a_season_turnovers,a_career_games_played,a_season_games_played
0,0,0,0,-6,129.63,103.3,0,-16,81,-1212,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,110.105263,1.0
1,0,0,0,-7,110.526,101.979,0,-16,57,-1162,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,70.894737,1.0
2,0,0,0,-7,143.836,119.66,0,-16,73,-1119,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,88.619048,1.0
3,0,0,1,-7,39.5349,98.5703,0,-16,86,-1259,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,102.409091,1.0
4,0,0,0,-9,103.704,99.3066,0,-16,54,-1298,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,85.809524,1.0
5,0,0,1,-6,95.9459,113.886,0,-16,74,-1109,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,113.380952,1.0
6,0,0,1,-12,38.1579,65.4646,0,-16,76,-1442,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,116.222222,1.0
7,0,0,0,-5,152.941,111.559,0,-16,51,-1142,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,89.863636,1.0
8,0,0,0,-9,145.161,97.6764,0,-16,62,-1162,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,60.100000,1.0


In [198]:
for col in combined.columns:
    print(col)

a_draws
a_draws_form
a_losses
a_losses_form
a_percentage
a_percentage_form
a_played
a_played_form
a_points_against
a_points_against_form
a_points_for
a_points_for_form
a_position
a_position_form
a_prem_points
a_prem_points_form
a_wins
a_wins_form
ateam
h_draws
h_draws_form
h_losses
h_losses_form
h_percentage
h_percentage_form
h_played
h_played_form
h_points_against
h_points_against_form
h_points_for
h_points_for_form
h_position
h_position_form
h_prem_points
h_prem_points_form
h_wins
h_wins_form
hteam
h_AFLfantasy
h_Supercoach
h_behinds
h_bounces
h_brownlow
h_clangers
h_clearances
h_contested_marks
h_contested_poss
h_disposals
h_frees_against
h_frees_for
h_goal_assists
h_goals
h_handballs
h_hitouts
h_inside50
h_kicks
h_marks
h_marks_in_50
h_one_percenters
h_rebound50
h_tackles
h_tog
h_uncontested_poss
h_centre_clearances
h_disposal_efficiency
h_effective_disposals
h_intercepts
h_metres_gained
h_stoppage_clearances
h_score_involvements
h_tackles_in_50
h_turnovers
h_next_matchid
h_career_

Can now use this to make predictions

In [199]:
X = combined[features]

In [200]:
X

,h_career_AFLfantasy,h_career_Supercoach,h_career_behinds,h_career_bounces,h_career_brownlow,h_career_clangers,h_career_clearances,h_career_contested_marks,h_career_contested_poss,h_career_disposals,...,a_played_form,a_wins_form,a_percentage_form,a_points_for_form,a_position_form,a_draws_form,h_career_games_played,h_season_games_played,a_career_games_played,a_season_games_played
0,71.282787,73.104673,0.383746,0.256715,0.072274,2.417926,1.541627,0.619073,6.442307,16.923070,...,-16,-9,103.3,-1252,-4,0,110.105263,1.0,110.105263,1.0
1,64.431558,68.905502,0.364790,0.221360,0.027855,2.228300,1.133752,0.500570,5.690617,14.917158,...,-16,-8,101.979,-1185,-7,0,70.894737,1.0,70.894737,1.0
2,67.485310,71.682558,0.385727,0.178535,0.052036,2.553424,1.559104,0.454911,6.499713,16.386184,...,-16,-8,119.66,-1339,-5,0,88.619048,1.0,88.619048,1.0
3,67.988297,69.975854,0.427136,0.398290,0.054524,2.444288,1.827762,0.507821,6.572994,16.626847,...,-16,-8,98.5703,-1241,-9,0,102.409091,1.0,102.409091,1.0
4,62.821298,68.312048,0.402674,0.139161,0.050016,2.333490,1.667395,0.488459,6.260419,14.880685,...,-16,-6,99.3066,-1289,-11,0,85.809524,1.0,85.809524,1.0
5,70.672632,73.946163,0.437596,0.350939,0.080972,2.356243,1.672995,0.373068,6.500349,17.357096,...,-16,-9,113.886,-1263,-6,0,113.380952,1.0,113.380952,1.0
6,67.156240,73.419904,0.397631,0.316018,0.116381,2.162723,1.761280,0.551069,6.648696,16.295610,...,-16,-3,65.4646,-944,-17,0,116.222222,1.0,116.222222,1.0
7,66.567581,69.924572,0.469336,0.335285,0.059219,2.269148,1.469908,0.400356,5.952193,15.908960,...,-16,-10,111.559,-1274,-3,0,89.863636,1.0,89.863636,1.0
8,64.447989,68.452068,0.370635,0.181727,0.051230,2.253393,1.280278,0.555795,5.886620,14.603728,...,-16,-6,97.6764,-1135,-10,0,60.100000,1.0,60.100000,1.0


In [205]:
grid_log_reg.decision_function(X)

array([[ -3.35632079, -12.15545438,   3.33830582],
       [ -1.39837606, -14.39455293,   1.35009815],
       [ -1.02777142, -12.55468551,   0.71060106],
       [ -1.00616815, -11.11658708,   0.89671882],
       [ -1.54335482, -12.16748738,   1.41125546],
       [ -2.32948484, -10.64132701,   2.21712385],
       [ -3.07058827, -10.27836424,   3.27470253],
       [ -1.78731017, -13.6710717 ,   1.65441458],
       [ -2.27894919, -13.41738615,   2.39943269]])

In [229]:
print(grid_log_reg.decision_function(X_test))

[[ 0.52516136 -4.66433299 -0.72350011]
 [-0.79354821 -4.10586978  0.69423111]
 [-0.15296358 -3.84293102  0.29424532]
 ...
 [-0.39704485 -3.68859956  0.33886572]
 [-0.05218006 -6.8555657   0.0714509 ]
 [-0.09048714 -6.65807857  0.21399919]]


In [234]:
test_data[['hteam','ateam','season']]

,hteam,ateam,season
13008,Sydney,St Kilda,2010
13006,Melbourne,Hawthorn,2010
13016,Essendon,Fremantle,2010
13019,Hawthorn,Geelong,2010
13022,North Melbourne,West Coast,2010
...,...,...,...
14925,Brisbane Lions,Gold Coast,2019
14923,Melbourne,Collingwood,2019
14934,Adelaide,Collingwood,2019
14932,Melbourne,Sydney,2019


In [206]:
grid_log_reg.predict(X)

array(['home', 'home', 'home', 'home', 'home', 'home', 'home', 'home',
       'home'], dtype=object)

In [209]:
fixture[fixture['round']=='2']

,Unnamed: 0,round,date,venue,hometeam,awayteam,homescore,awayscore,winner
9,9,2,26/03/2020 19:25,MCG,Collingwood,Richmond,NaN,NaN,NaN
10,10,2,27/03/2020 19:50,SCG,Sydney,Essendon,NaN,NaN,NaN
11,11,2,28/03/2020 13:45,MCG,Melbourne,Greater Western Sydney,NaN,NaN,NaN
12,12,2,28/03/2020 16:35,Marvel Stadium,Carlton,Footscray,NaN,NaN,NaN
13,13,2,28/03/2020 19:25,Gabba,Brisbane Lions,North Melbourne,NaN,NaN,NaN
14,14,2,28/03/2020 19:40,Adelaide Oval,Port Adelaide,Adelaide,NaN,NaN,NaN
15,15,2,29/03/2020 13:10,GMHBA Stadium,Geelong,Gold Coast,NaN,NaN,NaN
16,16,2,29/03/2020 15:20,Marvel Stadium,St Kilda,West Coast,NaN,NaN,NaN
17,17,2,29/03/2020 18:20,Optus Stadium,Fremantle,Hawthorn,NaN,NaN,NaN


In [222]:
combined[['hteam','ateam','h_percentage','a_percentage','h_wins','a_wins','h_season_Supercoach','a_season_Supercoach']]

,hteam,ateam,h_percentage,a_percentage,h_wins,a_wins,h_season_Supercoach,a_season_Supercoach
0,Collingwood,Richmond,252.941,129.63,1,1,89.473684,89.473684
1,Sydney,Essendon,104.225,110.526,1,1,80.736842,80.736842
2,Melbourne,Greater Western Sydney,65.3846,143.836,0,1,70.952381,70.952381
3,Carlton,Footscray,77.1429,39.5349,0,0,69.500000,69.500000
4,Brisbane Lions,North Melbourne,68.8889,103.704,0,1,68.666667,68.666667
5,Port Adelaide,Adelaide,262.069,95.9459,1,0,84.809524,84.809524
6,Geelong,Gold Coast,69.5238,38.1579,0,0,68.222222,68.222222
7,St Kilda,West Coast,96.4286,152.941,0,1,76.454545,76.454545
8,Fremantle,Hawthorn,90.4762,145.161,0,1,70.850000,70.850000


# Metadata and functions

In [7]:
from typing import Dict
import numpy as np

def get_season_rounds(matches: pd.DataFrame) -> Dict:
    """
    Return a dictionary with seasons as keys and number of games
    in season as values
    """
    seasons = matches['season'].unique()
    rounds_in_season = dict.fromkeys(seasons,0)
    
    for season in seasons:
        rounds_in_season[season] = max(matches[matches['season']==season]['round'])
    
    return rounds_in_season

# What teams participated in each season?
def get_season_teams(matches: pd.DataFrame) -> Dict:
    """
    Return a dictionary with seasons as keys and a list of teams who played
    in that season as values
    """
    seasons = matches['season'].unique()
    teams_in_season = {}

    for season in seasons:
        teams = list(matches[matches['season']==season]['hteam'].unique())
        teams.extend(list(matches[matches['season']==season]['ateam'].unique()))
        teams = np.unique(teams)
        teams_in_season[season] = list(teams)
        
    return teams_in_season

In [11]:
player_cols_to_agg = [
    'AFLfantasy',
    'Supercoach',
    'behinds',
    'bounces',
    'brownlow',
    'clangers',
    'clearances',
    'contested_marks',
    'contested_poss',
    'disposals',
    'frees_against',
    'frees_for',
    'goal_assists',
    'goals',
    'handballs',
    'hitouts',
    'inside50',
    'kicks',
    'marks',
    'marks_in_50',
    'one_percenters',
    'rebound50',
    'tackles',
    'tog',
    'uncontested_poss',
    'centre_clearances',
    'disposal_efficiency',
    'effective_disposals',
    'intercepts',
    'metres_gained',
    'stoppage_clearances',
    'score_involvements',
    'tackles_in_50',
    'turnovers'
]

match_cols = [
    'odds',
    'line'
]

ladder_columns = [
    'wins',
    'losses',
    'draws',
    'prem_points',
    'played',
    'points_for',
    'points_against',
    'percentage',
    'position'
]

misc_columns = [
    'games_played'
]

In [25]:
def get_upcoming_matches(matches, fixture, round_num=None):
    
    if round_num == None: # Get the latest populated round
        round_num = matches['round'].iloc[-1] + 1
        
    next_round = fixture[fixture['round']==str(round_num)]
    
    # Get list of home and away
    matches.sort_values(by=['season','round'],ascending=False,inplace=True)
    teams = list(next_round['hometeam'])
    teams = list(zip(teams,list(next_round['awayteam']))) # (home, away)
    
    # Initialise upcoming round
    df = pd.DataFrame()
    output = pd.DataFrame(columns = h_ladder_cols + h_ladder_form_cols + a_ladder_cols + a_ladder_form_cols + ['ateam','hteam'])
    
    # For each team, find the data that is relevant to them
    for team in teams:
        h_last_match = matches[(matches['hteam'] == team[0]) | (matches['ateam'] == team[0])].iloc[0]
        a_last_match = matches[(matches['hteam'] == team[1]) | (matches['ateam'] == team[1])].iloc[0]
        
        # Home team conditions, and use the 'game_cols' to update the ladder and ladder form for that team
        if team[0] == h_last_match['hteam']: # Home team was home team last game
            h_last_match_rel_cols = h_last_match[h_ladder_cols + h_ladder_form_cols + game_cols]
            h_last_match_rel_cols = update_ladder(h_last_match_rel_cols,'home')
        elif team[0] == h_last_match['ateam']: # Home team was away team last game
            h_last_match_rel_cols = h_last_match[a_ladder_cols + a_ladder_form_cols + game_cols]
            h_last_match_rel_cols = update_ladder(h_last_match_rel_cols,'away')
            
        # Away team conditions
        if team[1] == a_last_match['hteam']: # Away team was home team last game
            a_last_match_rel_cols = a_last_match[h_ladder_cols + h_ladder_form_cols + game_cols]
            a_last_match_rel_cols = update_ladder(a_last_match_rel_cols,'home')
        elif team[1] == a_last_match['ateam']: # Away team was away team last game
            a_last_match_rel_cols = a_last_match[a_ladder_cols + a_ladder_form_cols + game_cols]
            a_last_match_rel_cols = update_ladder(a_last_match_rel_cols,'away')
            
        h_last_match_rel_cols['hteam'] = team[0]
        a_last_match_rel_cols['ateam'] = team[1]
        
        # Make sure the columns are the right format
        h_col_final = []
        for col in h_last_match_rel_cols.index:
            if col[0] == 'h':
                h_col_final.append(col)
            else:
                col = 'h' + col[1:]
                h_col_final.append(col)
                
        a_col_final = []
        for col in a_last_match_rel_cols.index:
            if col[0] == 'a':
                a_col_final.append(col)
            else:
                col = 'a' + col[1:]
                a_col_final.append(col)   
        
        h_last_match_rel_cols.index = h_col_final
        a_last_match_rel_cols.index = a_col_final
            
        # Add all of these to the output.
        joined = pd.concat([h_last_match_rel_cols,a_last_match_rel_cols]).to_frame().T
        joined.drop('hscore',axis=1,inplace=True)
        joined.drop('ascore',axis=1,inplace=True)
        output = output.append(joined)
            
    matches.sort_values(by=['season','round'],ascending=True,inplace=True)
    return output

In [26]:
def update_ladder(last_match_rel_cols, last_game_h_a):
    if last_game_h_a == 'home':
        
        # Update wins, losses, draws and prem points
        if last_match_rel_cols['hscore'] > last_match_rel_cols['ascore']:
            last_match_rel_cols['h_wins'] = last_match_rel_cols['h_wins'] + 1
            last_match_rel_cols['h_wins_form'] = last_match_rel_cols['h_wins_form'] + 1
            last_match_rel_cols['h_prem_points'] = last_match_rel_cols['h_prem_points'] + 4
            last_match_rel_cols['h_prem_points_form'] = last_match_rel_cols['h_prem_points_form'] + 4
        elif last_match_rel_cols['hscore'] < last_match_rel_cols['ascore']:
            last_match_rel_cols['h_losses'] = last_match_rel_cols['h_losses'] + 1
            last_match_rel_cols['h_losses_form'] = last_match_rel_cols['h_losses_form'] + 1
        else:
            last_match_rel_cols['h_draws'] = last_match_rel_cols['h_draws'] + 1
            last_match_rel_cols['h_prem_points'] = last_match_rel_cols['h_prem_points'] + 2
            last_match_rel_cols['h_prem_points_form'] = last_match_rel_cols['h_prem_points_form'] + 2
            
        # Update points for and against
        last_match_rel_cols['h_points_for'] = last_match_rel_cols['h_points_for'] + last_match_rel_cols['hscore']
        last_match_rel_cols['h_points_against'] = last_match_rel_cols['h_points_against'] + last_match_rel_cols['ascore']
        last_match_rel_cols['h_points_for_form'] = last_match_rel_cols['h_points_for_form'] + last_match_rel_cols['hscore']
        last_match_rel_cols['h_points_against_form'] = last_match_rel_cols['h_points_against_form'] + last_match_rel_cols['ascore']
        
        # Update percentage
        last_match_rel_cols['h_percentage'] = (last_match_rel_cols['h_points_for'] / last_match_rel_cols['h_points_against']) * 100
        last_match_rel_cols['h_percentage_form'] = (last_match_rel_cols['h_points_for_form'] / last_match_rel_cols['h_points_against_form']) * 100
        
                
    if last_game_h_a == 'away':
        # Update wins, losses, draws and prem points
        if last_match_rel_cols['hscore'] > last_match_rel_cols['ascore']:
            last_match_rel_cols['a_losses'] = last_match_rel_cols['a_losses'] + 1
            last_match_rel_cols['a_losses_form'] = last_match_rel_cols['a_losses_form'] + 1
        elif last_match_rel_cols['hscore'] < last_match_rel_cols['ascore']:
            last_match_rel_cols['a_wins'] = last_match_rel_cols['a_wins'] + 1
            last_match_rel_cols['a_wins_form'] = last_match_rel_cols['a_wins_form'] + 1
            last_match_rel_cols['a_prem_points'] = last_match_rel_cols['a_prem_points'] + 4
            last_match_rel_cols['a_prem_points_form'] = last_match_rel_cols['a_prem_points_form'] + 4
        else:
            last_match_rel_cols['a_draws'] = last_match_rel_cols['a_draws'] + 1
            last_match_rel_cols['a_prem_points'] = last_match_rel_cols['a_prem_points'] + 2
            last_match_rel_cols['a_prem_points_form'] = last_match_rel_cols['a_prem_points_form'] + 2
            
        # Update points for and against
        last_match_rel_cols['a_points_for'] = last_match_rel_cols['a_points_for'] + last_match_rel_cols['ascore']
        last_match_rel_cols['a_points_against'] = last_match_rel_cols['a_points_against'] + last_match_rel_cols['hscore']
        last_match_rel_cols['a_points_for_form'] = last_match_rel_cols['a_points_for_form'] + last_match_rel_cols['ascore']
        last_match_rel_cols['a_points_against_form'] = last_match_rel_cols['a_points_against_form'] + last_match_rel_cols['hscore']
        
        # Update percentage
        last_match_rel_cols['a_percentage'] = (last_match_rel_cols['a_points_for'] / last_match_rel_cols['a_points_against']) * 100
        last_match_rel_cols['a_percentage_form'] = (last_match_rel_cols['a_points_for_form'] / last_match_rel_cols['a_points_against_form']) * 100
                
    return last_match_rel_cols

In [30]:
ladder_columns = {
    ('wins',0),
    ('losses',0),
    ('draws',0),
    ('prem_points',0),
    ('played',0),
    ('points_for',0),
    ('points_against',0),
    ('percentage',100),
    ('position',1)
}

ladder_cols = [i for i,j in ladder_columns]
h_ladder_cols = ['h_' + i for i,j in ladder_columns]
a_ladder_cols = ['a_' + i for i,j in ladder_columns]
h_ladder_form_cols = ['h_' + i + '_form' for i,j in ladder_columns]
a_ladder_form_cols = ['a_' + i + '_form' for i,j in ladder_columns]
h_ladder_form_cols_mapping = dict(zip(ladder_cols,h_ladder_form_cols))
a_ladder_form_cols_mapping = dict(zip(ladder_cols,a_ladder_form_cols))

game_cols = [
    'hscore',
    'ascore'
]

In [64]:
def update_last_game(df):
    for idx,row in df.iterrows():
        
        for col in cols_to_update:
            single_game_col = col[7:] # This is the non-aggregate column, e.g. 'Supercoach' instead of 'career_Supercoach'
            if col[0:7] == 'career_':
                df.at[idx,col] = (df.at[idx,single_game_col] + (df.at[idx,col] * (df.at[idx,'career_games_played']))) / df.at[idx,'career_games_played']
            elif col[0:7] == 'season_':
                df.at[idx,col] = (df.at[idx,single_game_col] + (df.at[idx,col] * (df.at[idx,'season_games_played']))) / df.at[idx,'season_games_played']
            else:
                raise Exception('Column not found, check what columns you\'re passing')
                
    return df

In [67]:
def get_player_names(image_string):
    """
    Returns the names of players who are named in a team
    """
    names = []
    name = ''
    i = 0
    while i <= len(image_string):
        if image_string[i] == ']':
            name = ''
            i += 2 # Skip the first space
        else:
            i += 1
            continue
        name = ''
        while (image_string[i] != ',') & (image_string[i] != '\n'):
            name += image_string[i]
            i += 1
            if i == len(image_string):
                break
        name = name.replace(' ','_')
        names.append(name)
        i += 1
    return names

In [89]:
def get_player_data(player_ids):
    last_games = pd.DataFrame(columns = players.columns)
    for player in player_ids:
        last_game_row = players[(players['playerid']==player) & (players['next_matchid'].isna())]
        last_games = last_games.append(last_game_row)
    return last_games

In [179]:
player_cols = ['AFLfantasy',
 'Supercoach',
 'behinds',
 'bounces',
 'brownlow',
 'clangers',
 'clearances',
 'contested_marks',
 'contested_poss',
 'disposals',
 'frees_against',
 'frees_for',
 'goal_assists',
 'goals',
 'handballs',
 'hitouts',
 'inside50',
 'kicks',
 'marks',
 'marks_in_50',
 'one_percenters',
 'rebound50',
 'tackles',
 'tog',
 'uncontested_poss',
 'centre_clearances',
 'disposal_efficiency',
 'effective_disposals',
 'intercepts',
 'metres_gained',
 'stoppage_clearances',
 'score_involvements',
 'tackles_in_50',
 'turnovers',
 'matchid',
 'next_matchid',
 'team',
 'career_AFLfantasy',
 'career_Supercoach',
 'career_behinds',
 'career_bounces',
 'career_brownlow',
 'career_clangers',
 'career_clearances',
 'career_contested_marks',
 'career_contested_poss',
 'career_disposals',
 'career_frees_against',
 'career_frees_for',
 'career_goal_assists',
 'career_goals',
 'career_handballs',
 'career_hitouts',
 'career_inside50',
 'career_kicks',
 'career_marks',
 'career_marks_in_50',
 'career_one_percenters',
 'career_rebound50',
 'career_tackles',
 'career_tog',
 'career_uncontested_poss',
 'career_centre_clearances',
 'career_disposal_efficiency',
 'career_effective_disposals',
 'career_intercepts',
 'career_metres_gained',
 'career_stoppage_clearances',
 'career_score_involvements',
 'career_tackles_in_50',
 'career_turnovers',
 'season_AFLfantasy',
 'season_Supercoach',
 'season_behinds',
 'season_bounces',
 'season_brownlow',
 'season_clangers',
 'season_clearances',
 'season_contested_marks',
 'season_contested_poss',
 'season_disposals',
 'season_frees_against',
 'season_frees_for',
 'season_goal_assists',
 'season_goals',
 'season_handballs',
 'season_hitouts',
 'season_inside50',
 'season_kicks',
 'season_marks',
 'season_marks_in_50',
 'season_one_percenters',
 'season_rebound50',
 'season_tackles',
 'season_tog',
 'season_uncontested_poss',
 'season_centre_clearances',
 'season_disposal_efficiency',
 'season_effective_disposals',
 'season_intercepts',
 'season_metres_gained',
 'season_stoppage_clearances',
 'season_score_involvements',
 'season_tackles_in_50',
 'season_turnovers',
 'career_games_played',
 'season_games_played']
